In [11]:
!pip install folium==0.12.1  
#Had to update the version of folium. 
import pandas as pd #for manipulating and cleaning the data
import folium #for visualising geospatial data

In [12]:
print(folium.__version__)

0.12.1


The Goal Is to Visualize Data through the day in New York

In [13]:
new_york = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/New York Visualisation Project/mta_1708.csv", error_bad_lines=False)

b'Skipping line 3356: expected 17 fields, saw 18\n'
b'Skipping line 59440: expected 17 fields, saw 18\nSkipping line 61296: expected 17 fields, saw 18\n'
b'Skipping line 66068: expected 17 fields, saw 18\nSkipping line 75328: expected 17 fields, saw 18\nSkipping line 81683: expected 17 fields, saw 18\nSkipping line 98179: expected 17 fields, saw 18\n'
b'Skipping line 116273: expected 17 fields, saw 18\n'
b'Skipping line 133094: expected 17 fields, saw 18\nSkipping line 137887: expected 17 fields, saw 18\nSkipping line 152688: expected 17 fields, saw 18\nSkipping line 160593: expected 17 fields, saw 18\n'
b'Skipping line 168801: expected 17 fields, saw 18\nSkipping line 170953: expected 17 fields, saw 18\nSkipping line 179203: expected 17 fields, saw 18\n'
b'Skipping line 201585: expected 17 fields, saw 18\nSkipping line 210727: expected 17 fields, saw 18\n'
b'Skipping line 289132: expected 17 fields, saw 18\n'
b'Skipping line 310840: expected 17 fields, saw 18\nSkipping line 311549: ex

In [14]:
new_york.shape

(6463520, 17)

In [15]:
#Let's understand the dataset a little bit
new_york.describe()

,DirectionRef,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop
count,6.463520e+06,6.378233e+06,6.378233e+06,6.449727e+06,6.449727e+06,6.463520e+06,6.463520e+06,6.456610e+06
mean,5.042095e-01,4.073109e+01,-7.393164e+01,4.072993e+01,-7.393229e+01,4.072987e+01,-7.393145e+01,2.251681e+02
std,4.999823e-01,8.982123e-02,9.237749e-02,8.952511e-02,9.306494e-02,8.637440e-02,8.741976e-02,9.873133e+02
min,0.000000e+00,4.050688e+01,-7.424695e+01,4.050811e+01,-7.424695e+01,4.050288e+01,-7.425234e+01,0.000000e+00
25%,0.000000e+00,4.066191e+01,-7.398702e+01,4.066086e+01,-7.398883e+01,4.066279e+01,-7.397902e+01,2.200000e+01
50%,1.000000e+00,4.071523e+01,-7.393245e+01,4.071336e+01,-7.393388e+01,4.072357e+01,-7.393584e+01,8.900000e+01
75%,1.000000e+00,4.080962e+01,-7.388094e+01,4.080755e+01,-7.387994e+01,4.080426e+01,-7.388561e+01,2.000000e+02
max,1.000000e+00,4.091237e+01,-7.370187e+01,4.091238e+01,-7.370139e+01,4.093372e+01,-7.370141e+01,3.592800e+04


In [16]:
new_york.columns

Index(['RecordedAtTime', 'DirectionRef', 'PublishedLineName', 'OriginName',
       'OriginLat', 'OriginLong', 'DestinationName', 'DestinationLat',
       'DestinationLong', 'VehicleRef', 'VehicleLocation.Latitude',
       'VehicleLocation.Longitude', 'NextStopPointName',
       'ArrivalProximityText', 'DistanceFromStop', 'ExpectedArrivalTime',
       'ScheduledArrivalTime'],
      dtype='object')

In [17]:
#For this project, we only need RecordedTime, VehicleRef, VehicleLocation.Latitiude, VehicleLocation.Longitude

new_york = new_york[['RecordedAtTime', #To get the position of the vehicle at a particular time
                     'VehicleRef', #To know exactly what vehicle we are talking about
                     'VehicleLocation.Latitude', #Position of the vehicle
       'VehicleLocation.Longitude'#Position of the vehicle at a given space
       ]]

In [18]:
new_york.head()

,RecordedAtTime,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude
0,2017-08-01 00:01:03,NYCT_7424,40.749403,-73.990841
1,2017-08-01 00:00:52,NYCT_406,40.651330,-73.938960
2,2017-08-01 00:01:18,NYCT_6449,40.706532,-73.804177
3,2017-08-01 00:01:05,NYCT_5846,40.770403,-73.917687
4,2017-08-01 00:01:05,NYCT_5846,40.770403,-73.917687


In [19]:
traffic = new_york.copy() #Rename the df appropriately


In [20]:
traffic.head()

,RecordedAtTime,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude
0,2017-08-01 00:01:03,NYCT_7424,40.749403,-73.990841
1,2017-08-01 00:00:52,NYCT_406,40.651330,-73.938960
2,2017-08-01 00:01:18,NYCT_6449,40.706532,-73.804177
3,2017-08-01 00:01:05,NYCT_5846,40.770403,-73.917687
4,2017-08-01 00:01:05,NYCT_5846,40.770403,-73.917687


In [ ]:
traffic.dtypes

RecordedAtTime                object
VehicleRef                    object
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
dtype: object

In [21]:
# We are going to divide the day up into hours to show the geospatioal visualisatn. The easiest way is by using datetime
# extracting the hour into a new column

traffic['RecordedAtTime'] = pd.to_datetime(traffic['RecordedAtTime'], format = "%Y-%m-%d %H:%M:%S")

In [23]:
#Create the hour column
traffic["Hour"] = traffic['RecordedAtTime'].apply(lambda x: x.hour+1)

In [24]:
traffic.dtypes

RecordedAtTime               datetime64[ns]
VehicleRef                           object
VehicleLocation.Latitude            float64
VehicleLocation.Longitude           float64
Hour                                  int64
dtype: object

In [ ]:
#We will be visualising the data for June 1st 2017
traffic = traffic[traffic['RecordedAtTime'].str.split(" ").apply(lambda x: x[0]== "2017-08-01")]

In [ ]:
traffic["RecordedAtTime"][1]

'2017-08-01 00:00:52'

In [ ]:
traffic.shape

(249945, 4)

In [ ]:
#drop the duplicated values
#find them first

traffic.duplicated().value_counts()

False    239132
True      10813
dtype: int64

In [ ]:
traffic.drop_duplicates(inplace=True)

In [ ]:
traffic.shape

(239132, 4)

In [ ]:
#We do not have any null values in the data, Thats's great
traffic.isnull().sum()

RecordedAtTime               0
VehicleRef                   0
VehicleLocation.Latitude     0
VehicleLocation.Longitude    0
dtype: int64

In [ ]:
traffic.head()

,RecordedAtTime,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,Hour
0,2017-08-01 00:01:03,NYCT_7424,40.749403,-73.990841,1
1,2017-08-01 00:00:52,NYCT_406,40.651330,-73.938960,1
2,2017-08-01 00:01:18,NYCT_6449,40.706532,-73.804177,1
3,2017-08-01 00:01:05,NYCT_5846,40.770403,-73.917687,1
6,2017-08-01 00:01:14,NYCT_3809,40.717332,-74.014027,1


In [ ]:
#Find the last location of every bus in an hour
# each vehicle ref is a new bus
df2=pd.DataFrame(traffic.groupby(["Hour", "VehicleRef"])["RecordedAtTime"].max())
df2.reset_index(inplace=True)
df2.head()

,Hour,VehicleRef,RecordedAtTime
0,1,NYCT_1209,2017-08-01 00:31:05
1,1,NYCT_1214,2017-08-01 00:51:49
2,1,NYCT_1218,2017-08-01 00:51:33
3,1,NYCT_1219,2017-08-01 00:51:28
4,1,NYCT_1222,2017-08-01 00:20:51


The goal of merging is to get the last time that a bus was recorded within an hour. i.e bus 1209 may have been recorded several times within an hour and bus 1214 may have been recorded at several different times. The goal is to find where they both are at a given time. 

We could have used the .min() function to find their position at the start of the hour

In [ ]:
df3 = pd.merge(df2, traffic, left_on=['Hour','VehicleRef','RecordedAtTime'],right_on=['Hour','VehicleRef','RecordedAtTime'])

In [ ]:
df3.head()

,Hour,VehicleRef,RecordedAtTime,VehicleLocation.Latitude,VehicleLocation.Longitude
0,1,NYCT_1209,2017-08-01 00:31:05,40.882927,-73.887798
1,1,NYCT_1214,2017-08-01 00:51:49,40.899238,-73.846127
2,1,NYCT_1218,2017-08-01 00:51:33,40.827203,-73.912274
3,1,NYCT_1219,2017-08-01 00:51:28,40.816870,-73.917410
4,1,NYCT_1222,2017-08-01 00:20:51,40.866586,-73.831376


In [ ]:
df3.shape

(48924, 5)

In [ ]:
lat_long_list = []
for i in range(1,25):
    temp=[]
    for index, instance in df3[df3['Hour'] == i].iterrows():
        temp.append([instance['VehicleLocation.Latitude'],instance['VehicleLocation.Longitude']])
    lat_long_list.append(temp)

At this point, the dataset is ready for visualisation 
We will be using a plugin from the folium library

In [ ]:
lat_long_list[1][1]

[40.836273999999996, -73.907379]

In [ ]:
from folium.plugins import HeatMapWithTime
#Declare the size of the figure
# fig1 = Figure(width=850,height=550)
map1=folium.Map(location=[40.712776, -74.005974],zoom_start=10)
# fig1.add_child(map1)

In [ ]:
hm = HeatMapWithTime(data= lat_long_list,
                     radius=5,
                     auto_play=True)



In [ ]:
hm

In [ ]:
hm.add_to(map1)

In [ ]:
map1